# Fast API 
  ##### Reference: https://fastapi.tiangolo.com/ko/
- Modern(현대적), fast(high-performance), **web framework for building APIs** 

- Requirements 
  - Python 3.6+
  - Starlette : for the web parts 
  - Pydantic  : for the data parts 

### Documnet everything with Open API
* Interactive API docs (대화형 문서)
  - 예시) http://192.168.1.199:8000/docs 

* Alternative API docs (대안문서)
  - 예시) http://192.168.1.199:8000/redoc

In [1]:
!pip install fastapi nest-asyncio pyngrok uvicorn

     |████████████████████████████████| 52 kB 978 kB/s 
     |████████████████████████████████| 745 kB 12.7 MB/s 
     |████████████████████████████████| 54 kB 3.5 MB/s 
     |████████████████████████████████| 60 kB 5.6 MB/s 
     |████████████████████████████████| 10.1 MB 40.4 MB/s 
     |████████████████████████████████| 54 kB 4.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=aad99af615ef0b6fdcbb1b530e0573bcf4b94f04f395e4c947e6c59f8df6f030
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [3]:
from typing import Optional
from fastapi import FastAPI

# Pydantic을 이용해 파이썬 표준 타입으로 분문을 선언 
from pydantic import BaseModel

import nest_asyncio
from pyngrok import ngrok
import uvicorn

### Step-by-Step 👣

#### Operation
- HTTP methods 
  - POST 
    - to create data 
    - @app.post()
  - GET
    - to read data 
    - @app.get()
  - PUT
    - to update data 
    - @app.put()
  - DELETE 
    - to delete data 
    - @app.delete()


In [5]:
app =FastAPI()

class Item(BaseModel):
  name: str
  price: float
  is_offer: Optional[bool] = None

@app.get('/')
def read_root():
    return {'message': 'Hello, World'}

@app.get('/items/{item_id}')
def read_item(item_id: int, q:Optional[str] = None):
    return{'item_id': item_id, 'q':q}

@app.put('/items/{item_id}')
def update_item(item_id: int, item: Item):
  return {'item_name': item.name, 'item_id': item_id}

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, host='0.0.0.0', port=8000)

Public URL: http://e13d-34-125-182-32.ngrok.io


INFO:     Started server process [63]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     114.203.149.236:0 - "GET / HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     114.203.149.236:0 - "GET / HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET / HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET / HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET / HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "PUT /items/2 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [63]


* Reference :https://github.com/ChristopherGS/ultimate-fastapi-tutorial/tree/main/part-4-pydantic-schemas

In [8]:
RECIPES = [ 
           { 
            "id": 1,
            "label": "Chicken Vesuvio",
            "source": "Serious Eats",
            "url": "http://www.seriouseats.com/recipes/2011/12/chicken-vesuvio-recipe.html",
           },
           {
            "id": 2,
            "label": "Chicken Paprikash",
            "source": "No Recipes",
            "url": "http://norecipes.com/recipe/chicken-paprikash/",
           },
           {
            "id": 3,
            "label": "Cauliflower and Tofu Curry Recipe",
            "source": "Serious Eats",
            "url": "http://www.seriouseats.com/recipes/2011/02/cauliflower-and-tofu-curry-recipe.html",
           },
]

In [9]:
%%writefile schemas.py

from pydantic import BaseModel, HttpUrl
from typing import Sequence


class Recipe(BaseModel):
    id: int
    label: str
    source: str
    url: HttpUrl


class RecipeSearchResults(BaseModel):
    results: Sequence[Recipe]


class RecipeCreate(BaseModel):
    label: str
    source: str
    url: HttpUrl
    submitter_id: int

Overwriting schemas.py


In [14]:
from fastapi import FastAPI, APIRouter, Query
from typing import Optional
from schemas import RecipeSearchResults, Recipe, RecipeCreate


api_router = FastAPI()

@api_router.get("/", status_code=200)
def root() -> dict:
    """
    Root GET
    """
    return {"msg": "Hello, World!"}


# Updated using to use a response_model
@api_router.get("/recipe/{recipe_id}", status_code=200, response_model=Recipe)
def fetch_recipe(*, recipe_id: int) -> dict:
    """
    Fetch a single recipe by ID
    """

    result = [recipe for recipe in RECIPES if recipe["id"] == recipe_id]
    if result:
        return result[0]


# Updated using the FastAPI parameter validation `Query` class
@api_router.get("/search/", status_code=200, response_model=RecipeSearchResults)
def search_recipes(
    *,
    keyword: Optional[str] = Query(None, min_length=3, example="chicken"),
    max_results: Optional[int] = 10
) -> dict:
    """
    Search for recipes based on label keyword
    """
    if not keyword:
        # we use Python list slicing to limit results
        # based on the max_results query parameter
        return {"results": RECIPES[:max_results]}

    results = filter(lambda recipe: keyword.lower() in recipe["label"].lower(), RECIPES)
    return {"results": list(results)[:max_results]}


# New addition, using Pydantic model `RecipeCreate` to define
# the POST request body
@api_router.post("/recipe/", status_code=201, response_model=Recipe)
def create_recipe(*, recipe_in: RecipeCreate) -> dict:
    """
    Create a new recipe (in memory only)
    """
    new_entry_id = len(RECIPES) + 1
    recipe_entry = Recipe(
        id=new_entry_id,
        label=recipe_in.label,
        source=recipe_in.source,
        url=recipe_in.url,
    )
    RECIPES.append(recipe_entry.dict())

    return recipe_entry


ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(api_router, host='0.0.0.0', port=8000)

Public URL: http://dacc-34-125-182-32.ngrok.io


INFO:     Started server process [63]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     114.203.149.236:0 - "GET / HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     114.203.149.236:0 - "GET / HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET / HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET /recipe/3 HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET /search/?keyword=chicken&max_results=10 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [63]


### Optional parameter 
- 함수 파라미터에 path가 정의되지 않은 변수를 정의할 경우, Query Parameter로 인식 
- Query는 'key-value'형식으로 되고, URL enldp `?`를 붙이고 여러개일 경우 `&` 사용 
- Default 는 `None`

### Request Body - Pydantic 
- Data validation and settings management using python type annotation 
- Class 함수 사용

In [ ]:
app = FastAPI()


# Optional parameter
course_items = [{"course_nmae": "Python"}, {"course_name" : "SQLAlchemy"},{"course_name": "NodeJS"}]
@app.get("/courses/")
def read_courses(start: int, end: int):
  return course_items[start : start + end]


# Data sent by the client to API 
# to declare one in Fast API, we can use Pydantic models
class Course(BaseModel):
  name: str
  description : Optional[str] = None
  price : int
  author : Optional[str] = None

@app.post("/courses/")
def create_course(course: Course):
  print('course name: '+ course.name)
  if course.description is not None:
    print('description: '+ course.description)
  print('price: ' + str(course))
  if course.author is not None:
    print('author: '+ course.author)
  return course